In [1]:
import pandas as pd  # pour la manipulation et l’analyse de données
import numpy as np  # pour les opérations numériques
from datetime import datetime  # pour manipuler les dates et heures

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split  # pour diviser les données en train et test
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
# from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix, classification_report, classification_report

import warnings  # pour gérer les messages d'avertissement
warnings.filterwarnings('ignore')  # ignorer les avertissements pour ne pas encombrer la sortie

In [2]:
# Importer le dataset

df1 = pd.read_excel('data/données_chèque_stage.xls')
df2 = pd.read_excel('data/données_LDC_stage.xls')

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


In [3]:
# Vérifier les premières lignes
df1.head()

# Vérifier les informations sur le dataset
df1.info()

# Vérifier les statistiques descriptives
df1.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50253 entries, 0 to 50252
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   NBRE         50213 non-null  float64       
 1   MCR          50213 non-null  float64       
 2   MDB          50213 non-null  float64       
 3   VALEUR       50253 non-null  int64         
 4   DATECPS      50253 non-null  datetime64[ns]
 5   NUM          50253 non-null  int64         
 6   MNT          49521 non-null  float64       
 7   ID           50253 non-null  int64         
 8   DNA          31752 non-null  object        
 9   PROVISIONNE  21391 non-null  float64       
 10  PROFESSION   41528 non-null  object        
 11  LIBCCL       50236 non-null  object        
 12  SORT         50253 non-null  object        
dtypes: datetime64[ns](1), float64(5), int64(3), object(4)
memory usage: 5.0+ MB


,NBRE,MCR,MDB,VALEUR,DATECPS,NUM,MNT,ID,PROVISIONNE
count,50213.000000,5.021300e+04,5.021300e+04,50253.0,50253,5.025300e+04,4.952100e+04,5.025300e+04,21391.0
mean,312.347639,4.417925e+06,5.428699e+06,30.0,2024-12-15 07:52:26.104710144,6.497059e+06,3.437451e+04,3.877509e+06,1.0
min,13.000000,0.000000e+00,1.149248e+03,30.0,2024-10-01 00:00:00,1.900000e+01,6.000000e-02,3.690000e+02,1.0
25%,54.000000,4.646420e+04,4.560294e+04,30.0,2024-11-04 00:00:00,6.351813e+06,4.656000e+02,1.097830e+06,1.0
50%,69.000000,9.110710e+04,9.124224e+04,30.0,2024-12-09 00:00:00,6.733913e+06,2.754200e+03,4.590973e+06,1.0
75%,141.000000,2.786869e+05,2.827489e+05,30.0,2025-01-15 00:00:00,7.000649e+06,4.986700e+03,6.192783e+06,1.0
max,113018.000000,2.541437e+09,2.537465e+09,30.0,2025-04-25 00:00:00,7.995979e+06,4.600000e+07,6.988269e+06,1.0
std,2121.423428,4.998041e+07,5.267235e+07,0.0,NaN,9.812790e+05,7.290768e+05,2.477617e+06,0.0


In [4]:
# Vérifier les premières lignes
df2.head()

# Vérifier les informations sur le dataset
df2.info()

# Vérifier les statistiques descriptives
df2.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8927 entries, 0 to 8926
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   NBRE        8864 non-null   float64       
 1   MCR         8864 non-null   float64       
 2   MDB         8864 non-null   float64       
 3   VALEUR      8927 non-null   int64         
 4   DATECPS     8927 non-null   datetime64[ns]
 5   NUM         0 non-null      float64       
 6   MNT         8925 non-null   float64       
 7   ID          8901 non-null   float64       
 8   DNA         8540 non-null   datetime64[ns]
 9   PROFESSION  8195 non-null   object        
 10  LIBCCL      8858 non-null   object        
 11  SORT        8927 non-null   object        
dtypes: datetime64[ns](2), float64(6), int64(1), object(3)
memory usage: 837.0+ KB


,NBRE,MCR,MDB,VALEUR,DATECPS,NUM,MNT,ID,DNA
count,8864.000000,8.864000e+03,8.864000e+03,8927.000000,8927,0.0,8925.000000,8.901000e+03,8540
mean,43.807310,4.827420e+04,4.738282e+04,40.996863,2025-01-09 08:47:38.384675840,NaN,1569.842667,8.754051e+10,1979-04-19 18:56:21.217798656
min,1.000000,0.000000e+00,0.000000e+00,40.000000,2024-10-01 00:00:00,NaN,24.500000,3.690000e+02,1900-06-20 00:00:00
25%,19.000000,5.423027e+03,5.247775e+03,41.000000,2024-12-02 00:00:00,NaN,90.700000,1.858184e+06,1970-11-18 00:00:00
50%,27.000000,9.717702e+03,8.963404e+03,41.000000,2025-01-16 00:00:00,NaN,143.430000,3.922730e+06,1980-11-02 00:00:00
75%,47.000000,1.282768e+04,1.309075e+04,41.000000,2025-02-28 00:00:00,NaN,500.000000,5.833789e+06,1987-07-06 12:00:00
max,1057.000000,4.486243e+06,4.458645e+06,41.000000,2025-03-28 00:00:00,NaN,75000.000000,1.881328e+13,2004-11-16 00:00:00
std,77.377669,3.178323e+05,3.150377e+05,0.055920,NaN,NaN,4883.869200,1.274948e+12,NaN


In [5]:
# Remplacer NaN dans la colonne PROVISIONNE par 0
df1['PROVISIONNE'] = df1['PROVISIONNE'].fillna(0).astype(int)

# Remplacer NaN dans la colonne PROFESSION par AUTRE PROFESSION
df1['PROFESSION'] = df1['PROFESSION'].fillna('AUTRE PROFESSION')
df1.isna().sum()

NBRE              40
MCR               40
MDB               40
VALEUR             0
DATECPS            0
NUM                0
MNT              732
ID                 0
DNA            18501
PROVISIONNE        0
PROFESSION         0
LIBCCL            17
SORT               0
dtype: int64

In [6]:
# Vérification du type des colonnes
df1['MCR'] = df1['MCR'].astype(str).str.replace(' ', '').str.replace(',', '.')
df1['MDB'] = df1['MDB'].astype(str).str.replace(' ', '').str.replace(',', '.')
df1['MNT'] = df1['MNT'].astype(str).str.replace(',', '.')

# Conversion en float
df1['MCR'] = pd.to_numeric(df1['MCR'], errors='coerce')
df1['MDB'] = pd.to_numeric(df1['MDB'], errors='coerce')
df1['MNT'] = pd.to_numeric(df1['MNT'], errors='coerce')

# Calculer l'âge à partir de la date de naissance (DNA)
df1['DNA'] = pd.to_datetime(df1['DNA'], errors='coerce')
df1['AGE'] = (pd.Timestamp.today() - df1['DNA']).dt.days // 365

In [8]:
# Remplacer NaN dans la colonne LIBCCL par Personne physique (la meme personne)
df1['LIBCCL'] = df1['LIBCCL'].fillna('Personne physique')
df1.isna().sum()

NBRE              40
MCR               40
MDB               40
VALEUR             0
DATECPS            0
NUM                0
MNT              732
ID                 0
DNA            18501
PROVISIONNE        0
PROFESSION         0
LIBCCL             0
SORT               0
AGE            18501
dtype: int64

In [9]:
# Encodage des variables catégorielles (PROFESSION, LIBCCL)
le_profession = LabelEncoder()  # Création de l'encodeur
df1['le_PROFESSION'] = le_profession.fit_transform(df1['PROFESSION'].fillna(''))

le_libccl = LabelEncoder()  # Création de l'encodeur pour LIBCCL
df1['le_LIBCCL'] = le_libccl.fit_transform(df1['LIBCCL'].fillna(''))

# Encoder la variable cible 'SORT'
df1['SORT'] = df1['SORT'].map({'P': 0, 'CNP': 1, 'AUTRE REJET': 1, 'PREAVIS': 1})

In [11]:
# Select only numeric columns
numeric_cols = df1.select_dtypes(include=['number']).columns

# Compute skewness only on numeric columns
skew_values = df1[numeric_cols].skew()

print("Skewness of numeric columns:")
print(skew_values)


Skewness of numeric columns:
NBRE             37.795877
MCR              42.440214
MDB              36.599130
VALEUR            0.000000
NUM              -2.963965
MNT              45.414844
ID               -0.304458
PROVISIONNE       0.300686
SORT              8.941069
AGE               0.318643
le_PROFESSION     7.857655
le_LIBCCL        -1.441373
dtype: float64


In [12]:
# Manual list from your skewness output
median_cols = ['NBRE', 'MCR', 'MDB', 'MNT', 'SORT', 'le_PROFESSION', 'le_LIBCCL', 'NUM']
mean_cols = ['VALEUR', 'ID', 'PROVISIONNE', 'AGE']

# Impute using median for skewed columns
for col in median_cols:
    df1[col] = df1[col].fillna(df1[col].median())
    print(f"{col}: Filled NaN with median (high skew)")

# Impute using mean for relatively normal columns
for col in mean_cols:
    df1[col] = df1[col].fillna(df1[col].mean())
    print(f"{col}: Filled NaN with mean (low skew)")


NBRE: Filled NaN with median (high skew)
MCR: Filled NaN with median (high skew)
MDB: Filled NaN with median (high skew)
MNT: Filled NaN with median (high skew)
SORT: Filled NaN with median (high skew)
le_PROFESSION: Filled NaN with median (high skew)
le_LIBCCL: Filled NaN with median (high skew)
NUM: Filled NaN with median (high skew)
VALEUR: Filled NaN with mean (low skew)
ID: Filled NaN with mean (low skew)
PROVISIONNE: Filled NaN with mean (low skew)
AGE: Filled NaN with mean (low skew)


In [ ]:
# Sélectionner les variables explicatives (X) et la cible (y)
X = df1[['NBRE', 'MCR', 'MDB', 'VALEUR', 'MNT', 'AGE', 'PROVISIONNE', 'le_PROFESSION', 'le_LIBCCL']]
y = df1['SORT']

In [ ]:
# Diviser les données en ensemble d'entraînement et de test, en stratifiant pour équilibrer les classes
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

In [ ]:
# client_table = cheque_overview[["ID","NBRE", "MCR", "MDB", "AGE", "PROFESSION", "LIBCCL"]]
# client_table.drop_duplicates(subset="ID", inplace=True)
# client_table.shape

# client_table.to_excel("data/client_table.xlsx", index=False)
